In [ ]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

In [ ]:
import os 

#first put a shortcut in your drive to the image processing folder


DATA_DIR = os.path.join('/content',
                        'drive',
                        'MyDrive',
                        'Image Processing and Analysis 2022',
                        'projects',
                        'Calcification Detection',
                        'dataset')

DATA_PREPROCESSED = os.path.join('/content',
                        'drive',
                        'MyDrive',
                        'Results',
                        'CLAHE+Dehazing')

RESULTS_DIR = os.path.join('/content',
                        'drive',
                        'MyDrive',
                        'Results', 
                        'proi_files',
                        'train-validation-test-224')


print(os.listdir(RESULTS_DIR))

results_file = os.listdir(RESULTS_DIR)


In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
#!pip install fastprogress
from fastprogress import master_bar, progress_bar
import gc
import time

In [ ]:
from google.colab.patches import cv2_imshow
#import plt for display
import matplotlib.pyplot as plt

#go into de directory of the images

# this have 3 outputs root directory, the folders in the path and the files in the path.
# we ignore _ the two first because we are not interested in those
_, _, images = next(os.walk(os.path.join(DATA_DIR,'images')))
_, _, breastMasks = next(os.walk(os.path.join(DATA_DIR,'masks')))
_, _, groundTruths = next(os.walk(os.path.join(DATA_DIR, 'groundtruths')))

images.sort()
breastMasks.sort()
groundTruths.sort()

# read numbers of normal images
normals = []
with open(os.path.join(DATA_DIR,'normals.txt')) as f:
    for line in f:
        normals.append(line[:-1])

In [ ]:
def sliding_window(image, stepSize, windowSize, groundTruth, mask):
    # slide a window across the image
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            # yield the current window
            # This yields the current window, label: 1 if MC and 0 if not MC, and Background: 1 if background, 0 if not background
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]], str(int(np.sum(groundTruth[y:y + windowSize[1], x:x + windowSize[0]])>0)), str(int(np.sum(mask[y:y + windowSize[1], x:x + windowSize[0]])==0)))

In [ ]:
def sliding_window_noCentered(image, stepSize, windowSize, groundTruth, mask):
    # slide a window across the image
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            # yield the current window
            # This yields the current window, label: 1 if MC and 0 if not MC, and Background: 1 if background, 0 if not background, roi of the groundTruth with CC
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]], str(int(np.sum(groundTruth[y:y + windowSize[1], x:x + windowSize[0]])>0)), str(int(np.sum(mask[y:y + windowSize[1], x:x + windowSize[0]])==0)), groundTruth[y:y + windowSize[1], x:x + windowSize[0]])

In [ ]:
import copy
import pandas as pd

groundTruthsDataFrame = pd.read_csv(os.path.join('/content',
                              'drive',
                              'MyDrive',
                              'Results',
                              'groundTruthStatsFinal.csv'))

groundTruthsDataFrame['key']= [i.split('_')[0] for i in groundTruthsDataFrame['name'].values.tolist()]

row_centroids = []
row_areas = []
for index, row in groundTruthsDataFrame.iterrows():
  count = 0
  centroids_final = []
  areas_final = []
  for area in eval(row['area']):
    if (area <= np.floor(np.pi*(15/2.0)**2)):
      areas_final.append(area)
      centroids_final.append(eval(row['centroids'])[count])
    count += 1
  
  row_centroids.append(centroids_final)
  row_areas.append(areas_final)
groundTruthsDataFrame['final_centroids'] = row_centroids
groundTruthsDataFrame['final_areas'] = row_areas


In [ ]:
groundTruthsDataFrame

In [ ]:
# groundTruthsDataFrame.loc[groundTruthsDataFrame.final_centroids]
normal_keys = []
for index, row in groundTruthsDataFrame.iterrows():
  if len(row.final_areas) == 0:
    normal_keys.append(row.key)

In [ ]:
normal_keys_all = []
for index, row in groundTruthsDataFrame.iterrows():
  if (row.num_labels == 0):
    normal_keys_all.append(row.key)

In [ ]:
normal_keys = [i for i in normal_keys if i != '']
len(normal_keys)

In [ ]:
print(len(normals)) # normals that professor gave us
print(len(normal_keys)) # normals no calcifications and images without big calcifications
print(len(normal_keys_all)) # normals that have something in the groundTruth (could be a big calcification)

In [ ]:
import pandas as pd

df_train_test = pd.read_csv(os.path.join('/content',
                              'drive',
                              'MyDrive',
                              'Results',
                              'standard_partitions.csv'), index_col=0)
train_keys = df_train_test.loc[df_train_test.partition == 'train']['image_id'].values
test_keys = df_train_test.loc[df_train_test.partition == 'test'].sample(frac=0.8, random_state=1)['image_id'].values
validation_keys= [i for i in df_train_test.loc[df_train_test.partition == 'test']['image_id'].values if i not in test_keys]


print(len(train_keys))
print(len(validation_keys))
print(len(test_keys))


## **For non centered MCs**

In [ ]:
from skimage.measure import label as label_ski
from skimage.measure import regionprops


windowsWH = [224]

for w in windowsWH:
  
    winW=w
    winH=w


    """    roi_width = xx; roi_height = xx; scale_factor = xx
        <image_filename> <px> <py>"""

    df_1 = pd.DataFrame(columns = [f'roi_width = {winW}', f' roi_height = {winH}', ' scale_factor = 1'])
    df_1.to_csv(os.path.join(RESULTS_DIR, f'noMC.w{winW}.train.proi'), index=False, sep=';')
    df_1.to_csv(os.path.join(RESULTS_DIR, f'noMC.w{winW}.validation.proi'), index=False, sep=';')
    df_1.to_csv(os.path.join(RESULTS_DIR, f'noMC.w{winW}.test.proi'), index=False, sep=';')
    df_1.to_csv(os.path.join(RESULTS_DIR, f'MC.w{winW}.train.proi'), index=False, sep=';')
    df_1.to_csv(os.path.join(RESULTS_DIR, f'MC.w{winW}.validation.proi'), index=False, sep=';')
    df_1.to_csv(os.path.join(RESULTS_DIR, f'MC.w{winW}.test.proi'), index=False, sep=';')

    df_2 = pd.DataFrame(columns = ['<image_filename>', '<px>', '<py>'])
    df_2.to_csv(os.path.join(RESULTS_DIR, f'noMC.w{winW}.train.proi'), index=False, mode='a', sep=' ')
    df_2.to_csv(os.path.join(RESULTS_DIR, f'noMC.w{winW}.validation.proi'), index=False, mode='a', sep=' ')
    df_2.to_csv(os.path.join(RESULTS_DIR, f'noMC.w{winW}.test.proi'), index=False, mode='a', sep=' ')
    df_2.to_csv(os.path.join(RESULTS_DIR, f'MC.w{winW}.train.proi'), index=False, mode='a', sep=' ')
    df_2.to_csv(os.path.join(RESULTS_DIR, f'MC.w{winW}.validation.proi'), index=False, mode='a', sep=' ')
    df_2.to_csv(os.path.join(RESULTS_DIR, f'MC.w{winW}.test.proi'), index=False, mode='a', sep=' ')

    already = []

    for image, breastMask, groundTruth in zip(progress_bar(images), breastMasks, groundTruths):
        start = time.time()

        mc_list = []
        no_mc_list = []


        count_neg = 0
        count_pos = 0
        count_bkg = 0


        # 20588020, 7717, 5328, 3787, 5725, 3859, 6934, 50995872
        # digits = '20587372'

        # if ((digits in image) and (digits in breastMask) and ('mask' in breastMask)):
        if image.split('_')[0] not in already:

            #if image not in already:
            print("image: ", image)
            #upload images
            img = cv2.imread(os.path.join(DATA_DIR,'images',image))
            imgMask = cv2.imread(os.path.join(DATA_DIR, 'masks', breastMask), cv2.IMREAD_GRAYSCALE)
            imgGroundTruth = cv2.imread(os.path.join(DATA_DIR, 'groundtruths', groundTruth), cv2.IMREAD_GRAYSCALE)


            gtRow = groundTruthsDataFrame.loc[groundTruthsDataFrame['key'] == image.split('_')[0]]

            if int(image.split('_')[0]) in list(train_keys):
              filename_MC = f'MC.w{winW}.train.proi'
              filename_noMC = f'noMC.w{winW}.train.proi'
            elif int(image.split('_')[0]) in list(test_keys):
              filename_MC = f'MC.w{winW}.test.proi'
              filename_noMC = f'noMC.w{winW}.test.proi'
            else:
              filename_MC = f'MC.w{winW}.validation.proi'
              filename_noMC = f'noMC.w{winW}.validation.proi'
            
            blobs = imgGroundTruth > 0.7 * imgGroundTruth.mean() #Thresholding the background
            blobs_labels, count = label_ski(blobs, background=0, return_num=True) #Getting labels of the connected components and the amount of them without considering the background
            props = regionprops(blobs_labels)

            # Label is 1 if MC and 0 in no MC. Background is  1 if background and 0 if not
            for (x, y, window, label, background, roi_labels) in sliding_window_noCentered(img, stepSize=int(winW/2), windowSize=(winW, winH), groundTruth=blobs_labels, mask=imgMask):

                if window.shape[0] != winH or window.shape[1] != winW:
                    continue

                if bool(int(background)):
                    count_bkg += 1
                    continue

                if label == '0':
                  no_mc_list.append([image, x, y])
                  count_neg += 1


                else: # unnecessary if label is 1 it will always be true
                  # https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops
                  count = 0
                  matching_labels = [i for i in np.unique(roi_labels) if i != 0]# not take into account background
                  
                  for match in matching_labels:
                    if props[match-1].area > np.floor(np.pi*(15/2.0)**2):
                      count+=1
                  if count == len(matching_labels):
                    # all matches correspond to big calcifications 
                    print('Area too big... ignored')
                  else:
                    mc_list.append([image, x, y])
                    count_pos += 1 
            print('File MC done')      
                
            df_mc = pd.DataFrame(mc_list,columns=['<image_filename>', '<px>', '<py>'])
            df_mc.to_csv(os.path.join(RESULTS_DIR, filename_MC), index=False, header=False, mode='a', sep=' ')



            df_no_mc = pd.DataFrame(no_mc_list,columns=['<image_filename>', '<px>', '<py>'])
            df_no_mc.to_csv(os.path.join(RESULTS_DIR, filename_noMC), index=False, header=False,  mode='a', sep=' ')

            print('File noMC done')

            print('Number of backgrounds ignored: ', count_bkg)
            print('Number of positives: ', count_pos)
            print('Number of negatives: ', count_neg)




            del df_no_mc, img, imgMask, imgGroundTruth, window, df_mc
            gc.collect()

            print("Time taken for the image: ", time.time() - start)

    # machine learning must be applied for the classification of the features extracted



# **For Centered MCs**

In [ ]:

windowsWH = [12, 24, 48, 96]

for w in windowsWH:
  
    winW=w
    winH=w


    """    roi_width = xx; roi_height = xx; scale_factor = xx
        <image_filename> <px> <py>"""

    df_1 = pd.DataFrame(columns = [f'roi_width = {winW}', f' roi_height = {winH}', ' scale_factor = 1'])
    df_1.to_csv(os.path.join(RESULTS_DIR, f'noMC.w{winW}.train.proi'), index=False, sep=';')
    df_1.to_csv(os.path.join(RESULTS_DIR, f'noMC.w{winW}.validation.proi'), index=False, sep=';')
    df_1.to_csv(os.path.join(RESULTS_DIR, f'noMC.w{winW}.test.proi'), index=False, sep=';')
    df_1.to_csv(os.path.join(RESULTS_DIR, f'MC.w{winW}.train.proi'), index=False, sep=';')
    df_1.to_csv(os.path.join(RESULTS_DIR, f'MC.w{winW}.validation.proi'), index=False, sep=';')
    df_1.to_csv(os.path.join(RESULTS_DIR, f'MC.w{winW}.test.proi'), index=False, sep=';')

    df_2 = pd.DataFrame(columns = ['<image_filename>', '<px>', '<py>'])
    df_2.to_csv(os.path.join(RESULTS_DIR, f'noMC.w{winW}.train.proi'), index=False, mode='a', sep=' ')
    df_2.to_csv(os.path.join(RESULTS_DIR, f'noMC.w{winW}.validation.proi'), index=False, mode='a', sep=' ')
    df_2.to_csv(os.path.join(RESULTS_DIR, f'noMC.w{winW}.test.proi'), index=False, mode='a', sep=' ')
    df_2.to_csv(os.path.join(RESULTS_DIR, f'MC.w{winW}.train.proi'), index=False, mode='a', sep=' ')
    df_2.to_csv(os.path.join(RESULTS_DIR, f'MC.w{winW}.validation.proi'), index=False, mode='a', sep=' ')
    df_2.to_csv(os.path.join(RESULTS_DIR, f'MC.w{winW}.test.proi'), index=False, mode='a', sep=' ')

    already = []

    for image, breastMask, groundTruth in zip(progress_bar(images), breastMasks, groundTruths):
        start = time.time()

        mc_list = []
        no_mc_list = []


        count_neg = 0
        count_pos = 0
        count_bkg = 0


        # 20588020, 7717, 5328, 3787, 5725, 3859, 6934, 50995872
        # digits = '20587372'

        # if ((digits in image) and (digits in breastMask) and ('mask' in breastMask)):
        if image.split('_')[0] not in already:

            #if image not in already:
            print("image: ", image)
            #upload images
            img = cv2.imread(os.path.join(DATA_DIR,'images',image))
            imgMask = cv2.imread(os.path.join(DATA_DIR, 'masks', breastMask), cv2.IMREAD_GRAYSCALE)
            imgGroundTruth = cv2.imread(os.path.join(DATA_DIR, 'groundtruths', groundTruth), cv2.IMREAD_GRAYSCALE)


            gtRow = groundTruthsDataFrame.loc[groundTruthsDataFrame['key'] == image.split('_')[0]]

            if int(image.split('_')[0]) in list(train_keys):
              filename_MC = f'MC.w{winW}.train.proi'
              filename_noMC = f'noMC.w{winW}.train.proi'
            elif int(image.split('_')[0]) in list(test_keys):
              filename_MC = f'MC.w{winW}.test.proi'
              filename_noMC = f'noMC.w{winW}.test.proi'
            else:
              filename_MC = f'MC.w{winW}.validation.proi'
              filename_noMC = f'noMC.w{winW}.validation.proi'

            for centroid in gtRow['final_centroids'].values[0]:
                if ((centroid[0]+(winW/2)) < img.shape[0]) and ((centroid[1]+(winW/2)) < img.shape[1]) and ((centroid[0]-(winW/2)) > 0) and ((centroid[1]-(winW/2)) > 0):
#                    plt.imshow(imgGroundTruth[int(centroid[0]-winW/2):int(centroid[0]+winW/2), int(centroid[1]-winW/2):int(centroid[1]+winW/2)], cmap='gray')
                    mc_list.append([image, int(centroid[1]-winW/2), int(centroid[0]-winW/2)])
                    count_pos += 1 
                
            df_mc = pd.DataFrame(mc_list,columns=['<image_filename>', '<px>', '<py>'])
            df_mc.to_csv(os.path.join(RESULTS_DIR, filename_MC), index=False, header=False, mode='a', sep=' ')
            
            print('File MC done')

            del df_mc
            
            # Label is 1 if MC and 0 in no MC. Background is  1 if background and 0 if not
            for (x, y, window, label, background) in sliding_window(img, stepSize=int(winW/3), windowSize=(winW, winH), groundTruth=imgGroundTruth, mask=imgMask):
                if window.shape[0] != winH or window.shape[1] != winW:
                    continue

                if bool(int(background)):
                    count_bkg += 1
                    continue

                if label == '0':
                  no_mc_list.append([image, x, y])
                  count_neg += 1

            df_no_mc = pd.DataFrame(no_mc_list,columns=['<image_filename>', '<px>', '<py>'])
            df_no_mc.to_csv(os.path.join(RESULTS_DIR, filename_noMC), index=False, header=False,  mode='a', sep=' ')

            print('File noMC done')

            print('Number of backgrounds ignored: ', count_bkg)
            print('Number of positives: ', count_pos)
            print('Number of negatives: ', count_neg)




            del df_no_mc, img, imgMask, imgGroundTruth, window
            gc.collect()

            print("Time taken for the image: ", time.time() - start)

    # machine learning must be applied for the classification of the features extracted



## Mean and standard deviation calculation

In [ ]:
count = 0

for image, breastMask in zip(images, breastMasks):
  if int(image.split("_")[0]) in test_keys:
    img = cv2.imread(os.path.join(DATA_PREPROCESSED,image))
    mask = cv2.imread(os.path.join(DATA_DIR,'masks',breastMask), cv2.IMREAD_GRAYSCALE) > 0
    maskedImg = img[mask]
    if count == 0:
      concatImages = maskedImg
      count += 1
    else:
      concatImages = np.concatenate((concatImages, maskedImg))

In [ ]:
totalmean = np.mean(concatImages)
totalstd = np.std(concatImages)
print(totalmean)
print(totalstd)